In [1]:
code_folder = '../codes'
import sys
sys.path.append(code_folder)

In [2]:
### Import packages
import numpy as np
from torch_geometric.nn import global_add_pool
from rdkit import RDLogger    
from torch_geometric.data import Data
RDLogger.DisableLog('rdApp.*')
from util_gnn import draw_graph


In [3]:
from torch_geometric.data import DataLoader
import torch

In [4]:
### Load data
batch_size = 64
mol_data = torch.load('qm9_smiles_noar.pt')
data_list = torch.load('qm9_data_noar.pt')
data_loader = DataLoader(data_list, batch_size=batch_size, shuffle=True, follow_batch=['edge_index', 'y'])

/home/miniconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [10]:
from gcn_model_sim_summ_2 import *
from ugcn_model_summ_2 import pre_GCNModel_edge_3eos


In [11]:
device = torch.device("cuda:1" if(torch.cuda.is_available()) else "cpu")


In [13]:
# Build discriminators

gcn_model = pre_GCNModel_edge_3eos(in_dim=10, \
                                   hidden_dim=128, \
                                   edge_dim=3, \
                                   edge_hidden_dim=32, \
                                   lin_hidden_dim=128, \
                                   out_hidden_dim=256, \
                                   device=device, \
                                    check_batch=None, 
                                    useBN=True, \
                                    droprate=0.3, 
                                    pool_method='trivial', \
                                    add_edge_link=False, 
                                    add_conv_link=True, \
                                    outBN=False, out_drop=0.3, 
                                    out_divide=4.0, 
                                    add_edge_agg=False, 
                                    real_trivial=False, 
                                    final_layers=2, 
                                    add_trivial_feature=False).to(device)

added_d = pre_GCNModel_edge_3eos(in_dim=10, \
                                   hidden_dim=128, \
                                   edge_dim=3, \
                                   edge_hidden_dim=32, \
                                   lin_hidden_dim=128, \
                                   out_hidden_dim=256, \
                                   device=device, \
                                    check_batch=None, 
                                    useBN=False, \
                                    droprate=0.3, 
                                    pool_method='trivial', \
                                    add_edge_link=False, \
                                    add_conv_link=False, \
                                    outBN=False, out_drop=0.3, 
                                    out_divide=4.0, 
                                    add_edge_agg=False, 
                                    real_trivial=True, 
                                    final_layers=2, 
                                    add_trivial_feature=True).to(device)
added_d.e1_ind = 0.0 # Not consider the mean of edges, just the sum


In [15]:
# Build generator

generator = UnpoolGeneratorQ(in_dim=256, \
                            edge_dim=3, 
                            node_dim=10, 
                            node_hidden_dim=[128, 128, 128, 64, 64], 
                            edge_hidden_dim=64, \
                            use_x_bn=True, 
                            use_e_bn=True, 
                            unpool_bn=True, 
                            link_bn=False, 
                            attr_bn=True, 
                            skip_z=True, 
                            skip_zdim=None, 
                            conv_type='nn', 
                            device=device, 
                            last_act='leaky', 
                            link_act='leaky', \
                            unpool_para=dict(add_perference=True, roll_bn=False, roll_simple=True, \
                                            add_additional_link=True), \
                             without_aroma=True).to(device)

In [16]:
from trainer import GANTrainer


In [18]:
train = GANTrainer(d=gcn_model, g=generator, \
                   rand_dim=256, train_folder='ULGAN_QM9', \
                   tot_epoch_num=200, eval_iter_num=1000, \
                   batch_size=64, \
                   device=device, d_add=added_d, \
                   learning_rate_g=2e-4, learning_rate_d=1e-4, \
                   lambda_g=10.0, \
                   max_train_G=2, \
                   tresh_add_trainG=0.2, \
                   use_loss='wgan', \
                   g_out_prob=True, \
                   lambda_rl=5e-3, lambda_nonodes = 0., 
                   lambda_noedges = 0., \
                   qm9=True, \
                   without_ar=True, \
                  )

In [ ]:
train.train(data_loader, verbose=False, use_data_x = 10, use_data_edgeattr=3, \
        evaluate_num=1000, mol_data=mol_data, alter_trainer=True, NN=200, reinforce_acclerate=True)

/home/miniconda3/envs/gnn/lib/python3.9/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/miniconda3/envs/gnn/lib/python3.9/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1/200][0/2079]	G Loss: -0.0002;D Loss: -0.0139; GP: 19.4810
now, we train G 1 times with (prob fake = 0.000, prob real = 0.015)
Mean x/edge attr:  tensor([0.2807, 0.2266, 0.2599, 0.2328, 0.3410, 0.3306, 0.3285, 0.3139, 0.3326,
        0.3534], grad_fn=<MeanBackward1>) tensor([0.3252, 0.3481, 0.3267], grad_fn=<MeanBackward1>) tensor(19.7500, grad_fn=<DivBackward0>)
size x/ some distribution: 7.515625 tensor([6., 7., 8., 8., 8., 7., 6., 6., 9., 6., 7., 9., 8., 7., 6., 6., 7., 7.,
        9., 8., 6., 9., 6., 9., 8., 7., 9., 9., 9., 9., 7., 6., 7., 8., 7., 9.,
        8., 6., 7., 9., 9., 9., 7., 8., 8., 9., 6., 6., 8., 9., 8., 6., 6., 8.,
        8., 7., 9., 7., 7., 6., 9., 7., 8., 6.])
Sample prob: -15.612592697143555
Validation, uniqueness, novelty:  (0.001, 1.0, 1.0)
[1/200][100/2079]	G Loss: 0.0488;D Loss: -0.2715; GP: 2.4253
now, we train G 2 times with (prob fake = -0.049, prob real = 0.220)
Mean x/edge attr:  tensor([0.6411, 0.1152, 0.1094, 0.1344, 0.7812, 0.1094, 0.1094, 0.7812, 0

Validation, uniqueness, novelty:  (0.007, 0.8571428571428571, 1.0)
[1/200][1100/2079]	G Loss: 0.0969;D Loss: -0.8880; GP: 0.4370
now, we train G 2 times with (prob fake = -0.097, prob real = 0.888)
Mean x/edge attr:  tensor([0.7000, 0.1462, 0.1538, 0.0000, 0.9577, 0.0327, 0.0096, 0.7885, 0.0615,
        0.1500], grad_fn=<MeanBackward1>) tensor([0.8484, 0.0901, 0.0615], grad_fn=<MeanBackward1>) tensor(18.0312, grad_fn=<DivBackward0>)
size x/ some distribution: 8.125 tensor([7., 7., 9., 9., 7., 8., 6., 7., 8., 9., 7., 8., 8., 9., 7., 9., 6., 7.,
        7., 9., 7., 9., 8., 9., 8., 9., 7., 9., 9., 9., 7., 8., 7., 8., 7., 7.,
        9., 7., 9., 6., 9., 9., 9., 9., 8., 8., 7., 9., 9., 9., 9., 9., 8., 9.,
        9., 9., 9., 9., 9., 7., 9., 8., 8., 9.])
Sample prob: -4.5749711990356445
[1/200][1200/2079]	G Loss: -0.2387;D Loss: -0.5814; GP: 0.3770
now, we train G 2 times with (prob fake = 0.239, prob real = 0.870)
Mean x/edge attr:  tensor([0.7098, 0.1314, 0.1588, 0.0000, 0.9588, 0.0353, 0.

[2/200][122/2079]	G Loss: -0.0817;D Loss: -0.3048; GP: 2.9606
now, we train G 2 times with (prob fake = 0.082, prob real = 0.171)
Mean x/edge attr:  tensor([0.7339, 0.1054, 0.1607, 0.0000, 0.9911, 0.0036, 0.0054, 0.7589, 0.1750,
        0.0661], grad_fn=<MeanBackward1>) tensor([0.8424, 0.1263, 0.0314], grad_fn=<MeanBackward1>) tensor(18.4375, grad_fn=<DivBackward0>)
size x/ some distribution: 8.75 tensor([7., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 7., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 7., 7., 9., 9., 8., 9., 9., 9., 9., 9., 9., 9., 7., 9., 9.,
        9., 9., 9., 9., 9., 9., 6., 9., 9., 9.])
Sample prob: -7.388788223266602
Validation, uniqueness, novelty:  (0.83, 0.8566265060240964, 0.9254571026722925)
[2/200][222/2079]	G Loss: 0.2517;D Loss: -0.6899; GP: 0.3718
now, we train G 2 times with (prob fake = -0.252, prob real = 0.360)
Mean x/edge attr:  tensor([0.6972, 0.1074, 0.1954, 0.0000, 0.9771, 

Validation, uniqueness, novelty:  (0.506, 0.8478260869565217, 0.8601398601398601)
[2/200][1222/2079]	G Loss: 0.1203;D Loss: -0.9433; GP: 0.1768
now, we train G 2 times with (prob fake = -0.120, prob real = 0.821)
Mean x/edge attr:  tensor([0.7148, 0.1604, 0.1248, 0.0000, 0.9590, 0.0178, 0.0232, 0.7879, 0.0713,
        0.1408], grad_fn=<MeanBackward1>) tensor([0.8546, 0.1420, 0.0034], grad_fn=<MeanBackward1>) tensor(18.3750, grad_fn=<DivBackward0>)
size x/ some distribution: 8.765625 tensor([9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 7., 9., 9., 7., 9., 9.,
        9., 9., 9., 9., 9., 9., 7., 9., 7., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 7., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 7., 9., 9., 7., 9.])
Sample prob: -5.210018157958984
[2/200][1322/2079]	G Loss: 0.2516;D Loss: -0.7915; GP: 0.2046
now, we train G 2 times with (prob fake = -0.252, prob real = 0.524)
Mean x/edge attr:  tensor([0.7325, 0.1167, 0.1508, 0.0000, 0

...

[5/200][488/2079]	G Loss: 0.2412;D Loss: -0.3836; GP: 0.0945
now, we train G 2 times with (prob fake = -0.241, prob real = 0.079)
Mean x/edge attr:  tensor([0.7531, 0.0894, 0.1538, 0.0036, 0.9821, 0.0179, 0.0000, 0.7800, 0.0859,
        0.1342], grad_fn=<MeanBackward1>) tensor([0.8624, 0.1063, 0.0313], grad_fn=<MeanBackward1>) tensor(18.9688, grad_fn=<DivBackward0>)
size x/ some distribution: 8.734375 tensor([9., 9., 9., 8., 9., 9., 8., 9., 9., 9., 9., 9., 8., 9., 9., 9., 8., 9.,
        8., 9., 8., 9., 8., 9., 8., 9., 8., 9., 8., 9., 9., 9., 9., 9., 8., 7.,
        9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 8., 9., 9., 8., 9.])
Sample prob: -7.327522277832031
Validation, uniqueness, novelty:  (0.549, 0.8706739526411658, 0.9267782426778243)
[5/200][588/2079]	G Loss: 0.3731;D Loss: -0.3397; GP: 0.0758
now, we train G 2 times with (prob fake = -0.373, prob real = -0.030)
Mean x/edge attr:  tensor([0.7092, 0.1684, 0.1170, 0.0053, 0.

Validation, uniqueness, novelty:  (0.841, 0.3626634958382878, 1.0)
[5/200][1588/2079]	G Loss: -0.0962;D Loss: -0.4696; GP: 0.1057
now, we train G 2 times with (prob fake = 0.096, prob real = 0.534)
Mean x/edge attr:  tensor([0.7452, 0.1274, 0.1274, 0.0000, 0.9651, 0.0332, 0.0017, 0.7225, 0.0628,
        0.2147], grad_fn=<MeanBackward1>) tensor([0.9038, 0.0612, 0.0351], grad_fn=<MeanBackward1>) tensor(19.1562, grad_fn=<DivBackward0>)
size x/ some distribution: 8.953125 tensor([9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 8., 9., 9., 9., 9.])
Sample prob: -5.270854949951172
[5/200][1688/2079]	G Loss: 0.4294;D Loss: -0.5865; GP: 0.1223
now, we train G 2 times with (prob fake = -0.429, prob real = 0.210)
Mean x/edge attr:  tensor([0.7365, 0.0995, 0.1640, 0.0000, 0.9791, 0.0175, 

[6/200][610/2079]	G Loss: -0.3673;D Loss: -0.5817; GP: 0.1421
now, we train G 2 times with (prob fake = 0.367, prob real = 0.967)
Mean x/edge attr:  tensor([0.7331, 0.1085, 0.1584, 0.0000, 0.9715, 0.0196, 0.0089, 0.7936, 0.0641,
        0.1423], grad_fn=<MeanBackward1>) tensor([0.8703, 0.0870, 0.0427], grad_fn=<MeanBackward1>) tensor(18.3125, grad_fn=<DivBackward0>)
size x/ some distribution: 8.78125 tensor([9., 9., 8., 9., 9., 8., 9., 9., 9., 8., 9., 9., 9., 9., 8., 8., 9., 9.,
        8., 9., 9., 9., 9., 8., 8., 9., 8., 9., 9., 9., 9., 9., 9., 9., 8., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9., 9., 9., 9., 8.,
        9., 8., 9., 9., 9., 9., 9., 9., 9., 8.])
Sample prob: -8.463835716247559
Validation, uniqueness, novelty:  (0.732, 0.7923497267759563, 0.9431034482758621)
[6/200][710/2079]	G Loss: -0.4370;D Loss: -0.3454; GP: 0.0916
now, we train G 2 times with (prob fake = 0.437, prob real = 0.746)
Mean x/edge attr:  tensor([0.7196, 0.1182, 0.1570, 0.0053, 0.97

Validation, uniqueness, novelty:  (0.774, 0.6188630490956072, 0.9144050104384134)
[6/200][1710/2079]	G Loss: 0.0552;D Loss: -0.8260; GP: 0.7924
now, we train G 2 times with (prob fake = -0.055, prob real = 0.517)
Mean x/edge attr:  tensor([0.7891, 0.0861, 0.1248, 0.0000, 0.9543, 0.0351, 0.0105, 0.7926, 0.0633,
        0.1441], grad_fn=<MeanBackward1>) tensor([0.8926, 0.0930, 0.0144], grad_fn=<MeanBackward1>) tensor(17.3125, grad_fn=<DivBackward0>)
size x/ some distribution: 8.890625 tensor([9., 9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9., 9., 9., 8., 9.,
        9., 9., 8., 9., 8., 9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 8., 9., 9., 9., 9., 9., 9., 9.,
        9., 9., 9., 9., 9., 9., 9., 8., 9., 9.])
Sample prob: -6.333161354064941
[6/200][1810/2079]	G Loss: 0.5022;D Loss: -1.1680; GP: 0.4951
now, we train G 2 times with (prob fake = -0.502, prob real = 0.642)
Mean x/edge attr:  tensor([0.7703, 0.0777, 0.1519, 0.0000, 0

[7/200][732/2079]	G Loss: 0.5731;D Loss: -1.4626; GP: 0.5709
now, we train G 2 times with (prob fake = -0.573, prob real = 0.975)
Mean x/edge attr:  tensor([0.7466, 0.1306, 0.0000, 0.1228, 0.8772, 0.1228, 0.0000, 0.8694, 0.0097,
        0.1209], grad_fn=<MeanBackward1>) tensor([0.8503, 0.0133, 0.1364], grad_fn=<MeanBackward1>) tensor(14.0938, grad_fn=<DivBackward0>)
size x/ some distribution: 8.015625 tensor([8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
        8., 8., 8., 8., 8., 8., 8., 8., 9., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
        8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
        8., 8., 8., 8., 8., 8., 8., 8., 8., 8.])
Sample prob: -1.1035795211791992
Validation, uniqueness, novelty:  (0.0, 0, 0)
[7/200][832/2079]	G Loss: -0.0116;D Loss: -1.4513; GP: 0.2438
now, we train G 2 times with (prob fake = 0.012, prob real = 1.403)
Mean x/edge attr:  tensor([0.8116, 0.0429, 0.1455, 0.0000, 0.9683, 0.0168, 0.0149, 0.7146, 0.0653

In [22]:
import os
candidate_evals = [(i,j ) for i,j in enumerate(train.evals) if j[0] > 0.9]
candidate_metrics = [np.prod(j) for i,j in enumerate(train.evals) if j[0] > 0.9]
choose_model = candidate_evals[np.argmax(candidate_metrics)][0]
generator = torch.load(os.path.join(train.folder, f'generator_{choose_model}.pt'))
torch.save(generator, 'ul_gan_qm9.pt')